In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append("../")

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [ ]:
import pandas as pd
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer

In [ ]:
# [pin]
file_path = "../data/data-hard.csv"
root_path = "../data/"


df = pd.read_csv(file_path)
df["prefix"] = "clsorg"
df = df.rename({"message": "input_text", "label": "target_text"}, axis=1)
df.sample(20)

,input_text,target_text,prefix
241,"""Москва. 3 апреля. INTERFAX.RU - Скидка россий...",112-3,clsorg
391,"""​Портфель из дивидендных акций РФ. Июль 2023....",111-5,clsorg
1753,#Идея 🧠 Закрытые идеи Мозгового центра Сигнало...,111-3,clsorg
3486,​Дивидендные акции. 21 месяц инвестирования. И...,111-4,clsorg
4579,❗️🇷🇺🇰🇿#SPBE #FRCH #брокеры #биржи #россия #каз...,255-4,clsorg
2087,Акционеры LKOH в моменте 👆,111-3,clsorg
1135,"""📋Итоги недели 🇷🇺🏦💰Ключевая ставка будет сниж...",88-3,clsorg
3691,"​​Индекс МосБиржи по итогам недели: -0,19% Ит...",90-3,clsorg
6284,📈 С подписчиками сайта обсудили на этой неделе...,237-3,clsorg
5955,💥🇷🇺#OKEY = +10% = макс за 7 мес,226-3,clsorg


In [ ]:
# m_name = "t5-small"
m_name = "cointegrated/rut5-small"
tokenizer = T5Tokenizer.from_pretrained(m_name)

In [ ]:
from src.t5.dataset import NERDataModel
from src.t5.model import NERModel

BATCH_SIZE = 128
EPOCHS = 10
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)
data_module = NERDataModel(train_df, test_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

In [ ]:
model = NERModel(lr=7e-4)

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="ner",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min",
)

trainer = Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=EPOCHS,
    accelerator="cuda",
)

In [ ]:
!rm -r lightning_logs

In [ ]:
trainer.fit(model, data_module)

In [ ]:
model.model.save_pretrained("./pretrained-rut5-2")

In [ ]:
trained_model = NERModel.load_from_checkpoint("checkpoints/ner-v5.ckpt")
trained_model.freeze()

In [ ]:
from final_solution.utils import generate_answer_batched
from src.t5.utils import evaluate_metric

predictions = generate_answer_batched(
    trained_model=trained_model, tokenizer=tokenizer, data=test_df, batch_size=64
)

In [ ]:
ldf = test_df.copy()
ldf["predictions"] = predictions
ldf[["tcomp", "tsent"]] = (
    ldf["target_text"].str.split(";", expand=True)[0].str.split("-", expand=True)
)
ldf[["pcomp", "psent"]] = (
    ldf["predictions"].str.split(";", expand=True)[0].str.split("-", expand=True)
)

In [ ]:
# [pin]

evaluate_metric(
    company_predictions=ldf["pcomp"].tolist(),
    company_labels=ldf["tcomp"].tolist(),
    sentiment_predictions=ldf["psent"].tolist(),
    sentiment_labels=ldf["tsent"].tolist(),
)

{'total': 57.80824034396828,
 'f1': 0.6391920745477017,
 'accuracy': 0.5169727323316639}